# Knn training sentences

in this notebook, we first extract each (training) sentence's sentence embedding, and use knn to find each sentence's NN

In [1]:
import os
import numpy as np
from read_data import InputExample,read_examples_from_file_knn
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
import pickle
import faiss

# conll2003

# 1 load data

In [2]:
num_examples = 14042
data_dir = '../data/conll2003'
train_examples = read_examples_from_file_knn(data_dir=data_dir,
                                         train_examples=num_examples,
                                         mode='train')

# 2 build sentence emb

In [13]:
train_examples[0:700][-1].words

['Jan',
 'Kroslak',
 '(',
 'Slovakia',
 ')',
 'vs.',
 'Chris',
 'Woodruff',
 '(',
 'U.S.',
 ')']

In [3]:
corpus = []
for example in train_examples:
    corpus.append(' '.join(example.words))

embedder = SentenceTransformer('bert-base-nli-mean-tokens')

corpus_embeddings = embedder.encode(corpus)

corpus_embeddings_ = np.array(corpus_embeddings)

# 3 knn search

In [4]:
# sent_id starts from 0
num_examples = 700
corpus_embeddings = corpus_embeddings_[0:num_examples]

In [5]:
d = 768
nb = corpus_embeddings.shape[0]
nq = corpus_embeddings.shape[0]
xb = corpus_embeddings
xq = corpus_embeddings

In [6]:
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
700


In [7]:
k = 20                          # we want to see 20 nearest neighbors
D, I = index.search(xq, k)     # actual search

# 4. save the index

In [8]:
data_dir = '../data/conll2003'
file_path = os.path.join(data_dir, 'sent_id_knn{}.pkl'.format(len(corpus_embeddings)))
pickle.dump( I, open( file_path, "wb" ) )   
file_path        

'../data/conll2003/sent_id_knn_700.pkl'